# Welcome to our Jupyter notebook!

Jupyter notebook is an amazing open-source tool that allows you to share live code, notes, equations, etc. in an interactive fashion. For each cell, you can specify whether you want it to be code or Markdown.

Insert a cell below in Markdown:

Insert cell below in code and run `print("Hello world")`

Okay let's start learning about pandas!

# Import modules that we'll need

To start, we'll load up a few modules with the `import` statement. When you load a module using import, all of the functions available are now accessible to you. Modules and import statements help programmers avoid naming conflicts because you can use short, straightforward names for functions and variables without worrying that they're already taken. Matlab does not have anything equivalent to Python's module system and therefore can be harder to read.

In [1]:
import sqlite3
import pandas as pd
import re
import numpy as np

# Getting and cleaning Data

### Download Data

Let's load the data up using the `read_csv` function from the Pandas package, which we've abbreviated as `pd`. By using `pd.read_csv`, we return what is called a pandas DataFrame. A DataFrame can be thought of as a 2D table, but the values within each of the columns must be the same datatype. For example, any entry in the Year column must be an integer, while an entry in the Cause column must be a string.

In [2]:
# Use a # to "comment out" anything in a code block - this is a nice way to take notes and document your code!

# Data sets available at https://catalog.data.gov/dataset
data = pd.read_csv('NCHS_-_Leading_Causes_of_Death__United_States.csv',',')


The first thing to do with our DataFrame is to look at the first few rows of the function using `head()`. We often do this just to confirm that we loaded the data correctly (that it has the correct column names).

In [3]:
# Look at data
data.head()

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
0,2012,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,21,2.6
1,2016,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.7
2,2013,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.8
3,2000,"Intentional self-harm (suicide) (*U03,X60-X84,...",Suicide,District of Columbia,23,3.8
4,2014,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Arizona,325,4.1


It looks just like our csv file!

### Explore the data using pandas

Instead of looking at the data in Excel, we can use methods properties within the pandas framework to describe basic features of our dataset (feel free to look at the pandas documentation on DataFrames if you are curious about the difference between a method and a property).

In [4]:
# Describe returns summary statistics on numerical variables
data.describe()

,Year,Deaths,Age-adjusted Death Rate
count,10296.000000,1.029600e+04,10296.000000
mean,2007.500000,1.536793e+04,128.037383
std,5.188379,1.121457e+05,224.381865
min,1999.000000,2.100000e+01,2.600000
25%,2003.000000,6.060000e+02,19.200000
50%,2007.500000,1.704500e+03,35.800000
75%,2012.000000,5.678000e+03,153.025000
max,2016.000000,2.744248e+06,1087.300000


In [5]:
# What are the column names?
data.columns

Index(['Year', '113 Cause Name', 'Cause Name', 'State', 'Deaths',
       'Age-adjusted Death Rate'],
      dtype='object')

In [6]:
# How many rows and columns are there?
data.shape

(10296, 6)

Note that the convention here is (rows, columns)

### Clean Data

Data typically needs to be "cleaned" before it's ready to use. For example, for pandas and other frameworks, spaces in names and names that start with numbers will cause problems. So first we will replace all spaces in column names with underscores and change column names that start with numbers:

In [7]:
# Clean names
def clean_names(df):
    L = []
    for col in df.columns:
        L.append(re.sub(r"\s+|-", '_', col))
    df.columns = L    

In [8]:
# Clean names and print out new names
clean_names(data)
print([col for col in data.columns])

['Year', '113_Cause_Name', 'Cause_Name', 'State', 'Deaths', 'Age_adjusted_Death_Rate']


In [9]:
#113 Cause Name will still cause trouble
cols = [col for col in data.columns]
cols[1] = 'Cause_Description'
data.columns = cols
print([col for col in data.columns])

['Year', 'Cause_Description', 'Cause_Name', 'State', 'Deaths', 'Age_adjusted_Death_Rate']


# Next we will create our database using SQLite and start making queries, but first let's talk a bit more about the fundamentals of databases and SQL 

# Making the database

In [ ]:
# Make year table
conn = sqlite3.connect("leading_cases_of_death.sqlite")
cur = conn.cursor()
# Sort list of unique years
yr_unique = np.sort(data.Year.unique()).tolist()
sql_statement1 = ("DROP TABLE IF EXISTS Year")
sql_statement2 = '''CREATE TABLE Year(
                    YearID INTEGER PRIMARY KEY,
                    Year INTEGER NOT NULL
                    )'''
cur.execute(sql_statement1)
cur.execute(sql_statement2)
for yr in yr_unique:
    cur.execute("INSERT INTO Year (Year) VALUES (?)", (yr,))
    conn.commit()
conn.close()


In [ ]:
# Check year table created
conn = sqlite3.connect("leading_cases_of_death.sqlite")
df_year = pd.read_sql_query("SELECT * FROM Year", conn)
conn.close()
df_year.head()

In [ ]:
# Create Cause Table
conn = sqlite3.connect("leading_cases_of_death.sqlite")
cur = conn.cursor()
# Create list of unique causes
cause_unique = data.Cause_Name.unique().tolist()
maxLengthCause = max([len(item) for item in cause_unique])
cause_desc_unique = data.Cause_Description.unique().tolist()
maxLengthDesc = max([len(item) for item in cause_desc_unique])
sql_statement1 = ("DROP TABLE IF EXISTS Cause")
sql_statement2 = '''CREATE TABLE Cause(
                    CauseID INTEGER PRIMARY KEY,
                    Cause_Name VARCHAR({0}),
                    Cause_Description VARCHAR({1})
                    )'''.format(maxLengthCause,maxLengthDesc)
cur.execute(sql_statement1)
cur.execute(sql_statement2)
for i in range(len(cause_unique)):
    cur.execute("INSERT INTO Cause (Cause_Name,Cause_Description) VALUES (?,?)", 
                (cause_unique[i], cause_desc_unique[i]))
    conn.commit()
conn.close()

In [ ]:
# Check cause table created
conn = sqlite3.connect("leading_cases_of_death.sqlite")
df_cause = pd.read_sql_query("SELECT * FROM Cause", conn)
conn.close()
df_cause.head()

In [ ]:
# Make state table
conn = sqlite3.connect("leading_cases_of_death.sqlite")
cur = conn.cursor()
# Sort list of unique years
state_unique = np.sort(data.State.unique()).tolist()
maxLength = max([len(item) for item in state_unique])
sql_statement1 = ("DROP TABLE IF EXISTS State")
sql_statement2 = '''CREATE TABLE State(
                    StateID INTEGER PRIMARY KEY,
                    State VARCHAR({0})
                    )'''.format(maxLength,)
cur.execute(sql_statement1)
cur.execute(sql_statement2)
for state in state_unique:
    cur.execute("INSERT INTO State (State) VALUES (?)", (state,))
    conn.commit()
conn.close()

In [ ]:
# Check state table created
conn = sqlite3.connect("leading_cases_of_death.sqlite")
df_state = pd.read_sql_query("SELECT * FROM State", conn)
conn.close()
df_state.head()

In [ ]:
# # Make deaths table
conn = sqlite3.connect("leading_cases_of_death.sqlite")
cur = conn.cursor()
# Merge ID values to dataframe
merged = pd.merge(data,df_state,how='outer',on=['State'])
merged = pd.merge(merged,df_cause,how='outer',on=['Cause_Name'])
merged = pd.merge(merged, df_year, how ='outer',on=['Year'])

sql_statement1 = ("DROP TABLE IF EXISTS Deaths")
sql_statement2 = '''CREATE TABLE Deaths (
                    ID INTEGER PRIMARY KEY,
                    YearID INTEGER,
                    CauseID INTEGER,
                    StateID INTEGER,
                    Deaths INTEGER,
                    Age_adjusted_Death_Rate FLOAT,
                    FOREIGN KEY (CauseID) REFERENCES Cause(CauseID),
                    FOREIGN KEY (YearID) REFERENCES Year(YearID),
                    FOREIGN KEY (StateID) REFERENCES State(StateID)
                    );'''
cur.execute(sql_statement1)
cur.execute(sql_statement2)

# Get data into right format
deaths = merged['Deaths'].values.tolist()
age_adj_rate = merged['Age_adjusted_Death_Rate'].values.tolist()
causeID = merged['CauseID'].values.tolist()
yearID = merged['YearID'].values.tolist()
stateID = merged['StateID'].values.tolist()

for i in range(len(merged)):
    cur.execute('''INSERT INTO Deaths 
    (Deaths,Age_adjusted_Death_Rate,CauseID,YearID,StateID) 
    VALUES (?,?,?,?,?)''',
    (deaths[i],age_adj_rate[i],causeID[i],yearID[i],stateID[i]))
    conn.commit()
conn.close()

In [ ]:
# Check death table created
conn = sqlite3.connect("leading_cases_of_death.sqlite")
df_death = pd.read_sql_query("SELECT * FROM Deaths", conn)
conn.close()
df_death.head()

# Querying the database

## Background

### About Relational Databases

Unlike an excel spreadsheet or a `pandas dataframe`, data is typically spread across multiple tables in a relational
database. The process of spreading data across multiple tables is called `normalization`. Normalization reduces
redundancies in the database (making the normalized database more compact in terms of disk space), makes it easier
(and safer) to change the value of a cell in a database, and can optimize the queries (or searches).

### About SQLite

Most relational databaes require a separate server process. This means you have to access the server in order to interact with the relational database. Here we're using `SQLite`, which is a relational database which has similar features to relational databases requiring servers (such as `MySQL`, `Postgre SQL`, or `SQL Server`), but doesn't require a server and is essentially plug-and-play. Most importantly, SQLite uses similar query language to the other databases. The language used to query all of these databases is based upon `SQL`, or `Structured Query Language`. We will sometimes point out equivalent commands for MySQL.

### Using Python to Interact With Relational Databases

`sqlite3` is a module within python that allows you to interact with `SQLite` within the comfort of python. Alternatively, you could interact with SQLite through a command line interface, but python makes it easier to run, store, and alter your queries. The `sqlite3` module with establish a connection with the sqlite3 database and assign this connection to an object, which we are calling `connection`:
```python
connection = sqlite3.connect("my_database")
```
We will use a nice feature of `pandas` which allows us to run a query and load it as a dataframe after we've opened up a connection with the database:
```python
result_df = pd.read_sql_query("My SQLite Query", connection)
```
The `pd` is how we tell the `pandas` module that we are talking to it, and the `read_sql_query` is the command we're giving the `pandas` module, which tells `pandas` to use the connection we've given it (inside of the paretheses) to run the query we've also given it, and ultimately to load the results of the query into a dataframe. Notice that I have to type the name of the dataframe at the end of the code block to see the results of the query. Finally, it is important to remember to close the database when you're done querying it, which you do by telling the connection to close:
```python
connection.close()
```
Lastly, do not worry if everything you just read sounded like gobble-dee-muck.  It's sufficient to just think of the commands as a series of spells that will work if you say them in the proper order.

## Getting to Know Your Database

One of the first things you'll want to do with a database with which you're unfamiliar is find out the names of the
`tables` within the database.

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", connection)
connection.close()
result_df

So our column names are `Year`, `Cause`, `State`, and `Deaths`. Notice that I have to type the name of the dataframe at the end of the code block to see the results of the query.

_As an aside:_ Although hopefully the command we used might make a little more sense as you continue with this notebook, it is fairly esoteric. In MySQL the equivalent command is
```SQL
SHOW tables;
```

Next you'll want to find out what are the `column` names in each table. We will do that table-by-table below, starting with the `Year` table.

_As an aside_: In MySQL you can use the command
```SQL
DESCRIBE my_table;
```

### Year Table

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT * FROM Year LIMIT 5", connection)
conn.close()
result_df

### Cause Table

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT * FROM Cause LIMIT 5", connection)
conn.close()
result_df

### State Table

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT * FROM State LIMIT 5", connection)
conn.close()
result_df

### Deaths Table

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT * FROM Deaths LIMIT 5", connection)
conn.close()
result_df

## Comparison to Original Dataframe

Lets take a look at our original dataframe, which we can do by typing its name `data`. We're going to once again use the `head` method to not overwhelm ourselves with the full table.

In [ ]:
data.head()

Comparing the original dataframe (which has a format identical to the `csv` file you earlier opened in `Excel` to the `Deaths` table, we see that the year, cause name, and state are recoded as numbers. This prevents Vermont from being stored in the database 1000's of times and instead a single number (47) is stored in the `Deaths` table. This is exactly what we meant by `normalization`: we've broken up the original table into several tables in order to store the data in a more efficient manner. We can figure out which state the number 47 refers to by looking in the `State` table. We can figure out which year the number 14 refers to by looking in the `Year` table. We can also figure out what cause of death the number 1 refers to by looking in the `Cause` table.

## Query Language Basics

Now we're going to introduce you to the SQLite query language. This will make some of the queries we ran in the section **Getting To Know Your Database** a little easier to understand, and we invite you to review that section after completing this one.

### The Select Statement

A very common SQL statement is the `SELECT` statement which allows you to select columns from a particular table:
```SQL
SELECT Column Names
FROM Table Names
```

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT Cause_Name FROM Cause", connection)
conn.close()
result_df

### The Limit Statement

If you don't want to see all of the output you can add a `LIMIT` expression:
```SQL
SELECT Column Names LIMIT n
FROM Table Names
```

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT Cause_Name FROM Cause LIMIT 3", connection)
conn.close()
result_df

### Selecting Multiple Columns

You can either select all of the columns by typing them out as a list, or by using `*`:
```SQL
SELECT * LIMIT n
FROM Table Names
```

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT Cause_Name, Cause_Description FROM Cause LIMIT 3", connection)
conn.close()
result_df

**Exercise**: Modify the query above so that you get the same result using `*` instead of the column names.

In [ ]:
# Answer exercise here

### The WHERE Statement

You can also specify a condition using a `WHERE` statement:
```SQL
SELECT Column Names
FROM Table Names
WHERE condition
```

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")
result_df = pd.read_sql_query("SELECT Cause_Description FROM Cause WHERE Cause_Name = 'CLRD'", connection)
conn.close()
result_df

**Exercise**: What is the description of unintential injuries?

In [ ]:
# Answer exercise here

### Single Joins

**Exercise**: Before we introduce the next class of SQL query, we'd like you to use `SELECT` and `WHERE` statements to identify the state, year, and cause in the first row of the `Deaths` table. _Hint: It will probably be easiest if you use three separate queries to answer this exercise._

In [ ]:
# Answer exercise here

Phew! That was a lot of work, and you had to remember what the CauseID, YearID, and StateID values were for the first row of the `Deaths` table (or at least copy and paste the values into your query). That's not very efficient and prone to errors. Computers to the rescue!  If you give your computer the right command, it will do all of this work for you.  The `JOIN` statement will do just the trick, and this type of statement begins with a `SELECT` statement:
```SQL
SELECT Table1.ColumnA, Table2.ColumnB
```
`Table1.ColumnA` means `ColumnA` from `Table1`, so we are telling the database to show us `ColumnA` from `Table1` and `ColumnB` from `Table2`. Now we will add the `JOIN` part:
```SQL
SELECT Table1.ColumnA, Table2.ColumnB
FROM Table1 
INNER JOIN Table2
ON Table1.ColumnC = Table2.ColumnD
```
The `FROM Table1` is code for saying, starting from `Table1`, and the `INNER JOIN Table 2 ON Table1.ColumnC = Table2.ColumnD` means make a table that has all of the rows from Table1 and Table2 where ColumnC and ColumnD have identical values.

_Aside_: There are many types of `JOINS`, but `INNER JOIN` is one of the most common.

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")

query = '''SELECT *
      FROM Deaths 
      LEFT OUTER JOIN State
      ON Deaths.StateID = State.StateID
      LIMIT 5;'''

result_df = pd.read_sql_query(query, connection)
conn.close()
result_df


The `INNER JOIN` combined the two tables.  It's now really easy to see that `StateID` number 47 refers to Vermont. Notice too that we assigned the query to a variable and gave that variable (`query`) as a a parameter to `pd.read_sql_query`. We also put `'''` around the query, which is a way of telling Python that we are writing out a query that we want to put on multiple lines. `SQLite` doesn't care if you break your query up into lines or not, but it sure makes things a lot easier to read! We could have used the command below and gotten exactly the same result:
```python
result_df = pd.read_sql_query("SELECT * FROM Deaths LEFT OUTER JOIN State ON Deaths.StateID = State.StateID LIMIT 5;", connection)
```

Let's repeat that query but this time only select the columns `State`, `Deaths`, and `Age_Adjusted_Death_Rate`.

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")

query = '''SELECT State.State, Deaths.Deaths, Deaths.Age_Adjusted_Death_Rate
      FROM Deaths 
      INNER JOIN State
      ON Deaths.StateID = State.StateID
      LIMIT 5;'''

result_df = pd.read_sql_query(query, connection)
conn.close()
result_df

**Exercise:** Repeat the above query, but this time create a table that contains `Year`, `Deaths`, and `Age_Adjusted_Death_Rate`

In [ ]:
# Answer exercise here

### Multiple Joins

Single Joins are great, but we need `Year`, `State`, and `Cause` joined with `Deaths` to be able to turn `Deaths` into something that we can understand with just a glance.

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")

query = '''SELECT Year.Year, State.State, Deaths.Deaths, Deaths.Age_Adjusted_Death_Rate
      FROM Deaths 
      INNER JOIN State
      ON Deaths.StateID = State.StateID
      INNER JOIN Year
      ON Deaths.YearID = Year.YearID
      LIMIT 5;'''

result_df = pd.read_sql_query(query, connection)
conn.close()
result_df

**Exercise:** Repeat the above query, but this time create a table that contains `Year`, `State`, `Cause`, `Deaths`, and `Age_Adjusted_Death_Rate`

In [ ]:
# Answer exercise here

### Joins with the WHERE Statement

We can add in a `WHERE` statement to select rows from the joined table that meet some condition.

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")

query = '''SELECT Year.Year, State.State, Cause.Cause_Name, Deaths.Deaths, Deaths.Age_Adjusted_Death_Rate
      FROM Deaths 
      INNER JOIN State
      ON Deaths.StateID = State.StateID
      INNER JOIN Year
      ON Deaths.YearID = Year.YearID
      INNER JOIN Cause
      ON Deaths.CauseID = Cause.CauseID
      WHERE YEAR = 2012
      LIMIT 5;'''

result_df = pd.read_sql_query(query, connection)
conn.close()
result_df

### The AND Statement

We can also add in an `AND` statment in the `WHERE` statement to select rows that meet two conditions

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")

query = '''SELECT Year.Year, State.State, Cause.Cause_Name, Deaths.Deaths, Deaths.Age_Adjusted_Death_Rate
      FROM Deaths 
      INNER JOIN State
      ON Deaths.StateID = State.StateID
      INNER JOIN Year
      ON Deaths.YearID = Year.YearID
      INNER JOIN Cause
      ON Deaths.CauseID = Cause.CauseID
      WHERE YEAR = 2012 AND Cause_Name = 'All causes'
      LIMIT 5;'''

result_df = pd.read_sql_query(query, connection)
conn.close()
result_df

### The ORDER BY Statement

The `ORDER BY` statement allows us to get results ordered by some column.  This makes it really easy to see the row with the highest of some value (or the lowest). The `ORDER BY` statement works like this:
```SQL
ORDER BY ColumnA ASC
```
This will give us a table that is sorted by ColumnA in ascending order. The statement below gives us the table sorted by ColumnA sorted in descending order.
```SQL
ORDER BY ColumnA DESC
```

In [ ]:
connection = sqlite3.connect("leading_cases_of_death.sqlite")

query = '''SELECT Year.Year, State.State, Cause.Cause_Name, Deaths.Deaths, Deaths.Age_Adjusted_Death_Rate
      FROM Deaths 
      INNER JOIN State
      ON Deaths.StateID = State.StateID
      INNER JOIN Year
      ON Deaths.YearID = Year.YearID
      INNER JOIN Cause
      ON Deaths.CauseID = Cause.CauseID
      WHERE YEAR = 2012 AND Cause_Name = 'All causes'
      ORDER BY State.State ASC
      LIMIT 5;'''

result_df = pd.read_sql_query(query, connection)
conn.close()
result_df

We've gotten the table in alphabetical order by State! Not very interesting! But you get the idea. You can use `ORDER By` with both numerical and alphabetical variables. For a much more interesting example of the `ORDER BY` statement, check out the exercise below.

**Exercise**: Which state had the highest Age-Adjusted Death Rate in 2012? Which state had the lowest?

# Wrapping Up

We hope you enjoyed this introduction to using `python`, `pandas` and `SQLite`! Our intention was to give you an introduction to these powerful software tools and to instill you with the confidence that _you_ can learn to use them! If you'd like to get some more information, we've provided you with some links with helpful tutorials for you to continue your learning.

[SQLite Tutorial](https://www.tutorialspoint.com/sqlite)
